In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# inspired by https://stackoverflow.com/questions/7391945/how-do-i-read-image-data-from-a-url-in-python
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from google.colab import drive
from tensorflow.keras.applications.inception_v3 import InceptionV3


drive.mount('/content/drive')
# Set the random seed
np.random.seed(1337)

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Year_5/ML-Artwork/data/

/content/drive/MyDrive/Year_5/ML-Artwork/data


In [12]:
(img_x, img_y) = (150, 150)
image_size=(img_x, img_y)
train_ds = keras.preprocessing.image_dataset_from_directory(
  "paintings_450",
  label_mode = "categorical",
  validation_split=0.3,
  subset="training",
  seed=1337,
  image_size=image_size,
  batch_size=20)

val_ds = keras.preprocessing.image_dataset_from_directory(
  "paintings_450",
  label_mode = "categorical",
  validation_split=0.3,
  subset="validation",
  seed=1337,
  image_size=image_size,
  batch_size=20)

Found 4411 files belonging to 10 classes.
Using 3088 files for training.
Found 4411 files belonging to 10 classes.
Using 1323 files for validation.


In [13]:
# we apply random horizontal flipping or small random rotations to augment the dataset since we only have 450 images.
# 
# data_augmentation = keras.Sequential(
#     [
#         layers.RandomFlip("horizontal"),
#         layers.RandomRotation(0.15),
#     ]
# )

# Apply `data_augmentation` to the training images.
# train_ds = train_ds.map(
#     lambda img, label: (data_augmentation(img), label),
#     num_parallel_calls=tf.data.AUTOTUNE,
# )
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [5]:
# help from https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/#:~:text=Pre-Trained%20Models%20for%20Image%20Classification%201%201.%20Very,first%20model%20coming%20from%20the%20ResNet%20family.%20

base_model = InceptionV3(input_shape=(img_x,img_y,3),
                   include_top=False, weights='imagenet')
for layer in base_model.layers:
  layer.trainable = False

87910968/87910968 [==============================] - 1s 0us/step


In [14]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(10, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.01),
              loss = 'binary_crossentropy', metrics = ['acc'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [15]:
train_ds = train_ds.repeat()
val_ds = val_ds.repeat()

In [16]:
model.fit(train_ds, validation_data=val_ds, steps_per_epoch=100,
          epochs=20, validation_steps=50)

Epoch 1/20
100/100 [==============================] - 231s 2s/step - loss: 357.5312 - acc: 0.1315 - val_loss: 0.6211 - val_acc: 0.1200
Epoch 2/20
100/100 [==============================] - 171s 2s/step - loss: 17.2092 - acc: 0.1172 - val_loss: 0.3836 - val_acc: 0.1060
Epoch 3/20
100/100 [==============================] - 173s 2s/step - loss: 0.9946 - acc: 0.1020 - val_loss: 0.3352 - val_acc: 0.1000
Epoch 4/20
100/100 [==============================] - 180s 2s/step - loss: 2.7587 - acc: 0.1071 - val_loss: 0.3304 - val_acc: 0.0950
Epoch 5/20
100/100 [==============================] - 176s 2s/step - loss: 2.2070 - acc: 0.0991 - val_loss: 0.3299 - val_acc: 0.1040
Epoch 6/20
100/100 [==============================] - 177s 2s/step - loss: 0.7965 - acc: 0.1005 - val_loss: 0.3295 - val_acc: 0.0920
Epoch 7/20
100/100 [==============================] - 178s 2s/step - loss: 0.7026 - acc: 0.0900 - val_loss: 0.3293 - val_acc: 0.0940
Epoch 8/20
100/100 [==============================] - 177s 2s/step